# SysML v2 Test Suite

This can notebook can be cleared using menu `Edit --> Clear All Outputs` and then all cells can be
run using menu `Run --> Run All Cells`.

**WARNING**: This file is generated, all modifications here will be ignored.


## Base Example Directory: ../../sysml/src/examples/Mass Roll-up Example

## File: MassConstraintExample.sysml

In [1]:
package MassConstraintExample {
	import ISQ::*;
	import SI::*;
	import ScalarFunctions::*;
	
	part def Engine {
		attribute m :> mass;
	}
	
	part def Transmission {
		attribute m :> mass;
	}
	
	part def Vehicle1 {
		attribute m : MassValue = eng::m + trans::m;
		
		part eng : Engine {
			attribute :>> m : MassValue;
		}
		
		part trans : Transmission {
			attribute :>> m : MassValue;
		}
	}
	
	part def Vehicle2 {
		assert constraint { m == eng::m + trans::m }
		
		attribute m : MassValue;
		
		part eng : Engine {
			attribute :>> m : MassValue;
		}
		
		part trans : Transmission {
			attribute :>> m : MassValue;
		}
	}
	
	constraint def MassConstraint3 (
		totalMass : MassValue, 
		partMasses : MassValue[0..*]) {
			
		totalMass == sum(partMasses)
	}
	
	part def Vehicle3 {
		assert constraint massConstraint : MassConstraint3 (
			totalMass = m, 
			partMasses = {eng::m, trans::m}
		);
		
		attribute m : MassValue;
		
		part eng {
			attribute m : MassValue;
		}
		
		part trans {
			attribute m : MassValue;
		}
	}
	
	constraint def MassConstraint4 (
		totalMass : MassValue, 
		partMasses : MassValue[0..*]);
	
	constraint mc : MassConstraint4 (
		totalMass : MassValue, 
		partMasses : MassValue[0..*]) {
		
		totalMass == sum(partMasses)
	}
	
	part def Vehicle4 {
		assert mc (
			totalMass = m, 
			partMasses = {eng::m, trans::m}
		);
		
		attribute m : MassValue;
		
		part eng : Engine {
			attribute :>> m : MassValue;
		}
		
		part trans : Transmission {
			attribute :>> m : MassValue;
		}
	}
	
	constraint def MassLimit (
		mass : MassValue, 
		maxMass : MassValue) {
			
		mass <= maxMass
	}
	
	part def Vehicle5 {
		assert constraint ml : MassLimit (
			mass = m, 
			maxMass = 2500@[kg]
		);
		
		attribute m : MassValue = eng::m + trans::m;
		
		part eng : Engine {
			attribute :>> m : MassValue;
		}
		
		part trans : Transmission {
			attribute :>> m : MassValue;
		}
	}	
	
}

Package MassConstraintExample (6fdbedff-7fd8-4afa-9186-e244e957ed3f)


## File: MassRollup.sysml

In [2]:
package MassRollup {
	import ScalarFunctions::*;
	
	part def MassedThing {
		attribute mass :> ISQ::mass; 
		attribute totalMass :> ISQ::mass;
	}
	
	part simpleThing : MassedThing {
		attribute totalMass redefines MassedThing::totalMass = mass;
	}
	
	part compositeThing : MassedThing {
		part subcomponents: MassedThing[*];
		
		attribute totalMass redefines MassedThing::totalMass =
			mass + (subcomponents->collect p:MassedThing (p::totalMass)->reduce '+' ?? 0.0); 
	}
	
	part filteredMassThing :> compositeThing {
		abstract attribute minMass :> ISQ::mass;
		
		attribute totalMass redefines MassedThing::totalMass =
			(./subcomponents/mass[p (p > minMass)])->reduce '+' ?? 0.0;
	}

}

Package MassRollup (c84539d5-5cc6-4ab6-b86f-3e2c7d47aafb)


## File: Vehicles.sysml

In [3]:
package VehicleMasses {
	import ScalarValues::*;
	import MassRollup::*;
	
	part def CarPart :> MassedThing {			
		attribute serialNumber: String;
	}
	
	part car: CarPart :> compositeThing {	
		attribute vin redefines serialNumber;
		
		part carParts: CarPart[*] redefines subcomponents;
		
		part engine :> simpleThing, carParts {
			//...
		}
		
		part transmission :> simpleThing, carParts {
			//...
		}
	}

	// Example usage
	import SI::*;	
	part c :> car {
		redefines mass = 1000@[kg];
		part redefines engine {
			redefines car::engine::mass = 100@[kg];
		}
		
		part redefines transmission {
			redefines car::engine::mass = 50@[kg];
		}	
	}
	
	// c::totalMass --> 1150.0@[kg]
}

Package VehicleMasses (b4436680-ee8a-44fd-8c29-5198668db350)
